In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from tqdm import tqdm

# GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터셋 로드
data = pd.read_csv('/data/ephemeral/home/data/yj_aug_train.csv')  # 데이터셋 파일 경로를 입력하세요

# 데이터셋 클래스 정의
class STSDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, is_train=True):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.is_train = is_train
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        sentence1 = str(self.df.loc[idx, 'sentence_1'])
        sentence2 = str(self.df.loc[idx, 'sentence_2'])
        
        inputs = self.tokenizer.encode_plus(
            sentence1,
            sentence2,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )
        
        input_ids = inputs['input_ids'].squeeze()  # [max_len]
        attention_mask = inputs['attention_mask'].squeeze()  # [max_len]
        
        item = {
            'input_ids': input_ids.to(device),
            'attention_mask': attention_mask.to(device),
        }
        
        if self.is_train:
            label = self.df.loc[idx, 'label']
            item['labels'] = torch.tensor(label, dtype=torch.float).to(device)
        
        return item

# 모델 클래스 정의
class RegressionModel(nn.Module):
    def __init__(self, model_name):
        super(RegressionModel, self).__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name, config=self.config)
        self.dropout = nn.Dropout(p=0.1)
        self.fc1 = nn.Linear(self.config.hidden_size, self.config.hidden_size)
        self.activation = nn.ReLU()
        self.regressor = nn.Linear(self.config.hidden_size, 1)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        if hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
            pooled_output = outputs.pooler_output
        else:
            pooled_output = outputs.last_hidden_state[:, 0]
        
        x = self.dropout(pooled_output)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.dropout(x)
        outputs = self.regressor(x)
        return outputs.squeeze()

# 모델 학습 함수 정의
def train(model, data_loader, optimizer, scheduler):
    model.train()
    losses = []
    for batch in tqdm(data_loader, desc='Training'):
        optimizer.zero_grad()
        outputs = model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask']
        )
        loss = criterion(outputs, batch['labels'])
        loss.backward()
        optimizer.step()
        scheduler.step()
        losses.append(loss.item())
    return np.mean(losses)



# 모델 평가 함수 정의
def evaluate(model, data_loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for batch in tqdm(data_loader, desc='Evaluating'):
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask']
            )
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(batch['labels'].cpu().numpy())
    predictions = np.array(predictions)
    actuals = np.array(actuals)
    pearson_corr = pearsonr(predictions, actuals)[0]
    return pearson_corr, predictions

# 데이터 분할 (학습/검증)
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(data, test_size=0.1, random_state=42)

# 토크나이저와 데이터셋 생성
max_len = 128
batch_size = 32

# model_names = [
#     'jhgan/ko-sroberta-multitask',
#     'BM-K/KoSimCSE-bert-multitask',
#     'snunlp/KR-SBERT-V40K-klueNLI-augSTS'
# ]

# model_names = [
#     'kykim/bert-kor-base',
#     'snunlp/KR-ELECTRA-discriminator',
#     'klue/roberta-small',
#     'beomi/KcELECTRA-base-v2022',
#     'monologg/koelectra-base-v3-discriminator',
#     'jhgan/ko-sroberta-multitask'
    
# ]

model_names = [
    'snunlp/KR-ELECTRA-discriminator',
    'beomi/KcELECTRA-base-v2022',
    'monologg/koelectra-base-v3-discriminator',
]

models = []
tokenizers = []
train_loaders = []
val_loaders = []

for model_name in model_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizers.append(tokenizer)
    
    train_dataset = STSDataset(train_df, tokenizer, max_len)
    val_dataset = STSDataset(val_df, tokenizer, max_len)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    train_loaders.append(train_loader)
    val_loaders.append(val_loader)

# 모델 학습 및 평가
num_epochs = 12
learning_rate = 2e-5

trained_models = []
val_predictions = []

for i, model_name in enumerate(model_names):
    print(f"\n=== 모델 {i+1}: {model_name} 학습 시작 ===")
    tokenizer = tokenizers[i]
    train_loader = train_loaders[i]
    val_loader = val_loaders[i]
    
    model = RegressionModel(model_name).to(device)
    
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps
    )
    criterion = nn.MSELoss()
    
    best_pearson = -1  # 최상의 Pearson 상관계수를 저장하기 위한 변수
    
    for epoch in range(num_epochs):
        print(f"\n에포크 {epoch+1}/{num_epochs}")
        train_loss = train(model, train_loader, optimizer, scheduler)
        print(f"훈련 손실: {train_loss:.4f}")
        
        pearson_corr, _ = evaluate(model, val_loader)
        print(f"검증 데이터 Pearson 상관계수: {pearson_corr:.4f}")
        
        # 모델 저장 (검증 성능이 향상될 때마다)
        if pearson_corr > best_pearson:
            best_pearson = pearson_corr
            torch.save(model.state_dict(), f'/data/ephemeral/home/yujin/results/new_model_{i+1}.pt')
            print(f"최고 성능 갱신, 모델 저장: model_{i+1}.pt")
    
    # 검증 데이터에 대한 예측 저장
    model.load_state_dict(torch.load(f'/data/ephemeral/home/yujin/results/new_model_{i+1}.pt'))  # 최적의 모델 로드
    _, preds = evaluate(model, val_loader)
    val_predictions.append(preds)
    
    trained_models.append(model)

# 앙상블 예측 (검증 데이터)
ensemble_preds = np.mean(val_predictions, axis=0)
actuals = val_df['label'].values

# 앙상블 결과 평가
ensemble_pearson = pearsonr(ensemble_preds, actuals)[0]
print(f"\n앙상블 모델의 Pearson 상관계수: {ensemble_pearson:.4f}")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



=== 모델 1: snunlp/KR-ELECTRA-discriminator 학습 시작 ===


/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



에포크 1/12


Training: 100%|██████████| 448/448 [01:38<00:00,  4.55it/s]


...

훈련 손실: 0.0469


Evaluating: 100%|██████████| 50/50 [00:14<00:00,  3.37it/s]


검증 데이터 Pearson 상관계수: 0.9818
최고 성능 갱신, 모델 저장: model_3.pt


Evaluating: 100%|██████████| 50/50 [00:14<00:00,  3.37it/s]


앙상블 모델의 Pearson 상관계수: 0.9867
